In [18]:
#import libaries
import pandas as pd
import numpy as np
import requests
import json
import os
import sqlalchemy
import time
from tqdm.notebook import tqdm
from io import StringIO
from google.cloud import storage

In [38]:
#extract data function
def extract_data(database_name, table):
    engine = sqlalchemy.create_engine(f'mysql+pymysql://root@localhost/{database_name}')
    query = f'SELECT * FROM {table}'
    chunk_size = 100000
    counter = 0
    #read by chunk
    for chunk in pd.read_sql_query(query, engine, chunksize=chunk_size):
        start_time = time.time()
        print(f'Processing Chunk {counter}...')
        counter += 1
        end_time = time.time()
        total_time = end_time - start_time
        print(f'Chunk {counter} processed in {total_time:.2f} seconds')
    return df

In [44]:
#function to store data to the google cloud
def store_to_google_bucket(df, df_name):
    try:
        service_account_key_path = 'different-state-hospital-price-fd662d2f48c2.json'
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key_path
        # Initialize the GCP storage client
        client = storage.Client()
        json_data = df.to_json(orient='records', lines=True)
        bucket_name = 'different-state-hospital-prices'
        destination_blob_name = f'{df_name}.json'
        # Get the bucket object
        bucket = client.get_bucket(bucket_name)
        # Create a new blob and upload the JSON data
        blob = bucket.blob(destination_blob_name)
        blob.upload_from_string(json_data, content_type='application/json')
        print(f'Data successfully uploaded to {destination_blob_name}')
    except Exception as e:
        print(f'An error occurred: {e}')

In [41]:
cpt_hcpcs_df = extract_data('hospital-price-transparency', 'cpt_hcpcs')
cpt_hcpcs_df.head()

Processing Chunk 0...
Chunk 1 processed in 0.00 seconds
Processing Chunk 1...
Chunk 2 processed in 0.00 seconds
Processing Chunk 2...
Chunk 3 processed in 0.00 seconds
Processing Chunk 3...
Chunk 4 processed in 0.00 seconds
Processing Chunk 4...
Chunk 5 processed in 0.00 seconds
Processing Chunk 5...
Chunk 6 processed in 0.00 seconds
Processing Chunk 6...
Chunk 7 processed in 0.00 seconds
Processing Chunk 7...
Chunk 8 processed in 0.00 seconds
Processing Chunk 8...
Chunk 9 processed in 0.00 seconds
Processing Chunk 9...
Chunk 10 processed in 0.00 seconds
Processing Chunk 10...
Chunk 11 processed in 0.00 seconds
Processing Chunk 11...
Chunk 12 processed in 0.00 seconds
Processing Chunk 12...
Chunk 13 processed in 0.00 seconds
Processing Chunk 13...
Chunk 14 processed in 0.00 seconds
Processing Chunk 14...
Chunk 15 processed in 0.00 seconds
Processing Chunk 15...
Chunk 16 processed in 0.00 seconds
Processing Chunk 16...
Chunk 17 processed in 0.00 seconds
Processing Chunk 17...
Chunk 18 p

,code,npi_number,payer,price
0,00000A,1053358010.0,CASH,75047.00
1,00000A,1336186394,CASH,75047.00
2,00001U,1003139775.0,CASH,457.23
3,00001U,1053824292.0,CASH,972.00
4,00001U,1417901406.0,CASH,296.00


In [ ]:
store_to_google_bucket(cpt_hcpcs_df, 'cpt_hcpcs')

In [ ]:
hospitals_df = extract_data('hospital-price-transparency', 'hospitals')
hospitals_df.head()

In [ ]:
store_to_google_bucket(hospitals_df, 'hospitals')

In [ ]:
prices_df = extract_data('hospital-price-transparency', 'prices')
prices_df.head()

In [ ]:
store_to_google_bucket(prices_df, 'prices')